# 손글씨 숫자 인식
## mnist 데이터 셋 구성
## 96~99p
 * 0~9까지 숫자 
 * train img-60,000장
 * test img-10,000장 * size-28*28
 * 1채널 gray scale 
 * 각 픽셀당 0~255 까지의 값을 취함
 * 각 이미지당 해당 실제 숫자 레이블 부여

In [14]:
import sys, os
sys.path.append(os.pardir) #부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist #github 자료에서 받아온 저자의  mnist.py 파일의 함수 (mnist파일을 로딩)
                                     #mnist.py 파일을 dataset 폴더 안에 존재, dataset 폴더는 현재 ipynb 파일과 함께 존재해야함

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

#각 데이터의 형상 출력
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)


(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [15]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28,28)
print(img.shape)

img_show(img)




5
(784,)
(28, 28)


# 신경망의 추론 처리
## 신경망 구성
 * 입력층 뉴런 784개(28pixels*28pixcels), 출력층 뉴런 10개(0-9)
 * 제1은닉층 : 뉴런 50개(임의값) , 제2은닉층 : 뉴련 100개(임의값)

In [27]:
import numpy as np
import pandas as pd

In [30]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y
    

def get_data():
    (x_train, t_train),(x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) +b3
    y = softmax(a3)
    
    return y

In [18]:
import pickle

In [31]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)): 
    y = predict(network, x[i])
    p = np.argmax(y) #가장 확률이 높은 원소의 인덱스를 취함
    if p == t[i]:
        accuracy_cnt += 1
        
print("Accuracy:" +str(float(accuracy_cnt)/len(x)))

Accuracy:0.9352


# 배치처리
 * 배치batch: 하나로 묶은 입력 데이터
 * 장점 : 한장당 처리 시간을 대폭 줄여 준다.
         -대부분 계산 라이브러리가 큰 배열 처리에 최적화되어 있음
         -버스에 주는 부하를 저감 ( 데이터 전송 병목 현상 저감)
 
 *  X(100 * 784) | W1(784 * 50) | W2(50 * 100) | W3(100 * 10) -> Y(100 * 10)
 *         (100개의 이미지를 하나의 batch로 묶어 처리)

In [33]:
x, t = get_data()
network = init_network()

batch_size = 100 #배치 사이즈
accuracy_cnt = 0

for i in range(0, len(x), batch_size): 
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    
    p = np.argmax(y_batch, axis=1) #가장 확률이 높은 원소의 인덱스를 취함
    accuracy_cnt += np.sum(p==t[i:i+batch_size])
    
print("Accuracy:" +str(float(accuracy_cnt)/len(x)))

Accuracy:0.9352
